<a href="https://colab.research.google.com/github/ameasure/colab_tutorials/blob/master/Finetune2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfer Learning with Finetune 

https://https://github.com/IndicoDataSolutions/finetune

In [1]:
!pip install -U finetune
#!python -m spacy download en_core_web_sm
#!python -m spacy link en_core_web_sm en
!pip install tables
!pip install xlrd
!wget 'https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx'

Requirement already up-to-date: finetune in /usr/local/lib/python3.6/dist-packages (0.6.7)
--2019-06-16 04:21:06--  https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ameasure/autocoding-class/master/msha.xlsx [following]
--2019-06-16 04:21:06--  https://raw.githubusercontent.com/ameasure/autocoding-class/master/msha.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4183086 (4.0M) [application/octet-stream]
Saving to: ‘msha.xlsx.3’

msha.xlsx.3         100%[===================>]   3.99M  --.-KB/s    in 0.08s   

2019-06-16 04:21:06

In [2]:
import pandas as pd

df = pd.read_excel('msha.xlsx')
df['ACCIDENT_YEAR'] = df['ACCIDENT_DT'].apply(lambda x: x.year)
df['ACCIDENT_YEAR'].value_counts()
df_train = df[df['ACCIDENT_YEAR'].isin([2010, 2011])][0:192].copy()
df_valid = df[df['ACCIDENT_YEAR'] == 2012][0:96].copy()
print('training rows:', len(df_train))
print('validation rows:', len(df_valid))

training rows: 192
validation rows: 96


In [0]:
from tensorflow import summary
%load_ext tensorboard.notebook

In [8]:
%tensorboard --logdir 'my_fine_tune'

Reusing TensorBoard on port 6015 (pid 1922), started 0:03:37 ago. (Use '!kill 1922' to kill it.)

In [7]:
!kill 1922

/bin/bash: line 0: kill: (1922) - No such process


In [5]:
from finetune import Classifier

# max_length is the maximum number of words we will use from each narrative
# behind the scenes finetune is setting aside 5% of the training data for validation
model = Classifier(batch_size=32, max_length=90, n_epochs=4, val_size=32,
                   tensorboard_folder='my_fine_tune')
model.fit(df_train['NARRATIVE'], df_train['INJ_BODY_PART'])

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
W0616 04:21:28.013924 140606581958528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/finetune/saver.py:66: The name tf.train.SessionRunHook is deprecated. Please use tf.estimator.SessionRunHook instead.

W0616 04:21:28.024330 140606581958528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/finetune/__init__.py:23: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0616 04:21:28.025609 140606581958528 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/finetune/__init__.py:23: The name tf.logging.ERROR is deprecated. Please use t

In [0]:
# generate preditions
df_valid['PREDICTED_PART'] = model.predict(df_valid['NARRATIVE'].values)
# look at a sample
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART']].sample(5)

In [0]:
pd.options.display.max_colwidth=500

df_valid['PROB_DICT'] = model.predict_proba(df_valid['NARRATIVE'].values)
df_valid[['NARRATIVE', 'PREDICTED_PART', 'PROB_DICT']].head()

In [0]:
# function that takes a row of our dataframe and returns the predicted probability
def get_probability(row):
    predicted_part = row['PREDICTED_PART']
    probability_dict = row['PROB_DICT']
    return probability_dict[predicted_part]

# apply get_probability to each row in our dataframe and store the result
df_valid['PREDICTED_PROB'] = df_valid.apply(func=get_probability, axis=1)
# take a peak at what we get
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART', 'PREDICTED_PROB']].sample(5).head()

In [0]:
from sklearn.metrics import accuracy_score, f1_score

mf1 = f1_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'], average='macro')
acc = accuracy_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'])
print('macro-f1:', mf1)
print('accuracy:', acc)